In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN

In [18]:
# Navigate to data folder and get file
project = os.path.realpath('./..')
datadir = os.path.join(project, 'data')
datafile = os.path.join(datadir, 'tweets_1M.json')

In [19]:
tweets = pd.read_json(datafile)

In [20]:
tweets.head()

,id,lat,lng,text,timeStamp,user_id
0,377652254096228352,37.446100,-121.883557,@Tanner_Cortez hey checkout the website: http:...,2013-09-11 04:38:08,224874450
1,377652255346159616,34.087406,-117.462604,@Lennayy7 😪,2013-09-11 04:38:08,312179473
2,377652262325456897,37.356131,-121.842867,i laugh a lot with that line,2013-09-11 04:38:10,54351774
3,377652264682655744,37.364664,-122.009629,sons of anarchy is back on woop woop,2013-09-11 04:38:11,343219606
4,377652271116722176,37.382600,-121.995000,Drinking a Fresh Squeezed IPA by @deschutesbee...,2013-09-11 04:38:12,1569395935


In [21]:
# Save as HDF5 file for speed of access
h5file = os.path.join(datadir, 'tweets_1M.h5')
tweets.to_hdf(h5file, 'tweets', mode='w')

/home/paul/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->axis0] [items->None]

  f(store)
/home/paul/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block0_items] [items->None]

  f(store)
/home/paul/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->unicode,key->block1_items] [items->None]

  f(store)
/home/paul/anaconda2/lib/python2.7/site-packages/pandas/io/pytables.py:260: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-ty